## 使用本地模型
## 使用ollama的llama3.2

In [13]:
from crawl4ai.extraction_strategy import JsonCssExtractionStrategy
from crawl4ai import LLMConfig

# Generate a schema (one-time cost)
#html = "<div class='product'><h2>Gaming Laptop</h2><span class='price'>$999.99</span></div>"
html = "<div class='item'><h2>Item 1</h2><a href='https://example.com/item1'>Link 1</a></div>"

# Or using Ollama (open source, no token needed)
schema = JsonCssExtractionStrategy.generate_schema(
    html,
    llm_config = LLMConfig(provider="ollama/llama3.2", api_token=None)  # Not needed for Ollama
)

# Use the schema for fast, repeated extractions
strategy = JsonCssExtractionStrategy(schema)

In [14]:
## 模型要大一點的
schema

{'name': 'Item List',
 'baseSelector': '.item',
 'baseFields': [{'name': 'data_href',
   'type': 'attribute',
   'attribute': 'href'}],
 'fields': [{'name': 'title', 'selector': 'h2', 'type': 'text'},
  {'name': 'link', 'selector': 'a', 'type': 'attribute', 'attribute': 'href'}]}

In [ ]:
from crawl4ai.extraction_strategy import JsonCssExtractionStrategy
from crawl4ai import LLMConfig

# Generate a schema (one-time cost)
html = "<div class='product'><h2>Gaming Laptop</h2><span class='price'>$999.99</span></div>"

# Using OpenAI (requires API token)
schema = JsonCssExtractionStrategy.generate_schema(
    html,
    llm_config = LLMConfig(        
        provider="gemini/gemini-2.5-flash",
        api_token="your-gemini-token")  # Required for OpenAI
)
# Use the schema for fast, repeated extractions
strategy = JsonCssExtractionStrategy(schema)